### Approximate H2 molecule, evaluate Hamiltonian term by term

In [2]:
import qsharp
import qsharp.azure

Preparing Q# environment...


In [4]:
qsharp.reload()

Reloading workspace.

In [5]:
from Microsoft.Quantum.Chemistry.Hamiltonian import GetHamiltonianTermH2

In [6]:
GetHamiltonianTermH2.estimate_resources(
    nOp=11
)

{'CNOT': 4,
 'QubitClifford': 12,
 'R': 0,
 'Measure': 5,
 'T': 0,
 'Depth': 0,
 'Width': 5,
 'QubitCount': 5,
 'BorrowedWidth': 0}

We have 11 qubits on the IonQ system so this is feasible to run!

In [7]:
rid = ""

In [8]:
qsharp.azure.connect(
   resourceId=rid,
   location="West US")
qsharp.azure.target("ionq.simulator")

Connected to Azure Quantum workspace TestGuen in location westus.
Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


{'id': 'ionq.simulator', 'current_availability': 'Available', 'average_queue_time': 0}

In [9]:
n_op = 0
result = qsharp.azure.execute(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}")

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 0
   Job ID: ae8a3f47-57a2-46ea-8fd2-4e8f3163f97c
Waiting up to 30 seconds for Azure Quantum job to complete...
[5:55:13 PM] Current job status: Succeeded


In [10]:
result

{'1': 0.9999999999999998}

Loop over all Hamiltonian terms

In [12]:
from Microsoft.Quantum.Samples.Chemistry.SimpleVQE.Utils import ExpandedCoefficients

In [13]:
from qdk.chemistry.broombridge import load_and_encode

Adding package microsoft.quantum.chemistry.jupyter.

In [14]:
encoded_data_h2 = load_and_encode("../data/broombridge/hydrogen_0.2.yaml")

In [38]:
coeffs = []
for term_type, terms in enumerate(encoded_data_h2[1]):
    for (spin_orbitals, coeff) in terms:
        coeffs += ExpandedCoefficients(coeff=coeff, termType=term_type)
coeffs

[0.17120128499999998,
 0.17120128499999998,
 -0.222796536,
 -0.222796536,
 0.1686232915,
 0.12054614575,
 0.16586802525,
 0.16586802525,
 0.12054614575,
 0.1743495025,
 0.0,
 0.0,
 -0.0453218795,
 -0.0453218795,
 0.0,
 0.0,
 0.0453218795,
 0.0453218795]

Start job for each term that has a nonzero coefficient

In [40]:
jobs = [
    qsharp.azure.submit(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0
]

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 0
   Job ID: 81b11124-f5e9-4612-8343-9153e823cd86
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 1
   Job ID: a7524bab-33fb-49bf-b879-e2bd9e810535
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 2
   Job ID: d689907e-0347-4290-8c6b-b40531b5d32d
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 3
   Job ID: 4c13487f-6a39-4cde-8ca9-74a2af671aee
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.simulator...
Job succ

In [42]:
results = [qsharp.azure.output(j.id) for j in jobs]

In [43]:
results

[{'1': 0.9999999999999998},
 {'1': 0.9999999999999998},
 {'0': 0.9999999999999998},
 {'0': 0.9999999999999998},
 {'0': 0.9999999999999998},
 {'1': 0.9999999999999998},
 {'1': 0.9999999999999998},
 {'1': 0.9999999999999998},
 {'1': 0.9999999999999998},
 {'0': 0.9999999999999998},
 {'0': 0.4999999999999998, '1': 0.4999999999999998},
 {'0': 0.4999999999999998, '1': 0.4999999999999998},
 {'0': 0.4999999999999998, '1': 0.4999999999999998},
 {'0': 0.4999999999999998, '1': 0.4999999999999998}]

Calculate energy

In [46]:
def calc_energy(coeffs, results):
    return sum([(2. * res.get("0", 0.0) - 1.) * coeff for coeff, res in zip(coeffs, results)])

In [47]:
calc_energy(coeffs, results)

-1.0178511900000002

In [48]:
qsharp.azure.target("ionq.qpu")

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


{'id': 'ionq.qpu', 'current_availability': 'Available', 'average_queue_time': 12252}

In [49]:
jobs_hw = [
    qsharp.azure.submit(GetHamiltonianTermH2, nOp=n_op, shots=1000, jobName=f"Hamiltonian term {n_op}") 
    for n_op, coeff in enumerate(coeffs) if coeff != 0
]

Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 0
   Job ID: 7c8839d0-a553-44db-88a1-1c4591d19196
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 1
   Job ID: d73bcfc3-aefc-4a9f-8fc5-e71b60a74d30
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 2
   Job ID: bfd41d0e-96ca-4583-9aee-6425e76eddf8
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 shots.
   Job name: Hamiltonian term 3
   Job ID: e6f663c6-f23f-4db1-9973-ab83f6fdeaf0
Submitting Microsoft.Quantum.Chemistry.Hamiltonian.GetHamiltonianTermH2 to target ionq.qpu...
Job successfully submitted for 1000 sh

In [54]:
[qsharp.azure.status(j.id).status for j in jobs_hw]

['Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting',
 'Waiting']

### Todo's

* Make this work "offline" without rerunning on HW
* Add energy calculation and compare to value from QPE
* Minimize number of cells to execute